# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bhuban,20.8833,85.8333,26.85,87,100,1.39,IN,1696184126
1,1,dudinka,69.4058,86.1778,-14.07,100,70,1.79,RU,1696184126
2,2,papatowai,-46.5619,169.4708,10.26,82,92,4.17,NZ,1696184126
3,3,iqaluit,63.7506,-68.5145,4.85,75,75,2.06,CA,1696184126
4,4,massawa,15.6097,39.4500,32.81,53,4,2.05,ER,1696184127


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"]<27) & (city_data_df["Max Temp"]>21) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values

city_data_df = city_data_df.dropna()
# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,bethel,41.3712,-73.4140,25.20,62,0,4.12,US,1696184131
53,53,vila velha,-20.3297,-40.2925,23.97,81,0,4.12,BR,1696184134
71,71,albany,42.6001,-73.9662,25.51,62,0,1.34,US,1696184084
126,126,renala khurd,30.8833,73.6000,25.09,20,0,1.54,PK,1696184144
131,131,colonia,50.9333,6.9500,21.64,68,0,3.09,DE,1696183891
146,146,pisco,-13.7000,-76.2167,21.68,78,0,4.12,PE,1696184147
182,182,el abiodh sidi cheikh,32.8930,0.5484,26.68,23,0,3.85,DZ,1696184153
253,253,rio grande,-32.0350,-52.0986,22.06,65,0,3.43,BR,1696184164
261,261,primero de mayo,-32.2552,-58.4228,22.68,22,0,2.88,AR,1696184165
282,282,saint-pierre,-21.3393,55.4781,22.82,73,0,2.57,RE,1696184168


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
37,bethel,US,41.3712,-73.4140,62,
53,vila velha,BR,-20.3297,-40.2925,81,
71,albany,US,42.6001,-73.9662,62,
126,renala khurd,PK,30.8833,73.6000,20,
131,colonia,DE,50.9333,6.9500,68,
146,pisco,PE,-13.7000,-76.2167,78,
182,el abiodh sidi cheikh,DZ,32.8930,0.5484,23,
253,rio grande,BR,-32.0350,-52.0986,65,
261,primero de mayo,AR,-32.2552,-58.4228,22,
282,saint-pierre,RE,-21.3393,55.4781,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    # name_address = 
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
vila velha - nearest hotel: Hotel Prainha
albany - nearest hotel: No hotel found
renala khurd - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
pisco - nearest hotel: La Portada
el abiodh sidi cheikh - nearest hotel: No hotel found
rio grande - nearest hotel: Hotel Vila Moura Executivo
primero de mayo - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
crucecita - nearest hotel: No hotel found
westport - nearest hotel: Norwalk Inn and Conference Center
laguna - nearest hotel: Holiday Inn Express & Suites
monroeville - nearest hotel: DoubleTree by Hilton Hotel Pittsburgh - Monroeville Convention Center
lucas - nearest hotel: Hawthorn Suites by Wyndham Holland/Toledo Area
rockland - nearest hotel: Cullens Hotel
pulsano - nearest hotel: Sud


,City,Country,Lat,Lng,Humidity,Hotel Name
37,bethel,US,41.3712,-73.4140,62,Hampton Inn Danbury
53,vila velha,BR,-20.3297,-40.2925,81,Hotel Prainha
71,albany,US,42.6001,-73.9662,62,No hotel found
126,renala khurd,PK,30.8833,73.6000,20,No hotel found
131,colonia,DE,50.9333,6.9500,68,Wasserturm Hotel Cologne
146,pisco,PE,-13.7000,-76.2167,78,La Portada
182,el abiodh sidi cheikh,DZ,32.8930,0.5484,23,No hotel found
253,rio grande,BR,-32.0350,-52.0986,65,Hotel Vila Moura Executivo
261,primero de mayo,AR,-32.2552,-58.4228,22,No hotel found
282,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols=["Hotel Name", "Country"]
)

# # Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)